In [33]:
with open("7-2-2024-22-30-50.replay", 'r') as f:
    content = f.read()

import json
json_strings = content.strip().split('\n')
parsed_data = []
for json_str in json_strings:
    try:
        parsed_data.append(json.loads(json_str))
    except:
        print("Error parsing JSON")

In [34]:
%load_ext autoreload
%autoreload 2

from termite.game import TerminalGame, Player
from termite.units import Scout, Demolisher, Interceptor, Turret, Support, Wall

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
# Format: 
# Default keys: [p1Units, p1Stats, p2Units, p2Stats, events, turnInfo]
# Frame 1 is unique (start of the game). All keys are different from the default keys: [debug, unitInformation, timingAndReplay, resources, SeasonCompatibilityModeP1, SeasonCompatibilityModeP2]
# Frame -1 is unique (end of the game). There's the additional key endStats.

from collections import defaultdict

start_info = parsed_data[0]
end_info = parsed_data[-1]
frames = parsed_data[1:-1]

spawn_events = {}
spawn_map = {0: "wall",
            1: "support",
            2: "turret",
            3: "scout",
            4: "demolisher",
            5: "interceptor",
            7: "upgrade"}

for frame in frames:
    in_active_phase, round_idx, frame_in_round, global_frame_idx = frame["turnInfo"]
    in_active_phase = bool(in_active_phase)
    if not in_active_phase:
        # Deployment phase
        for spawn_event in frames[1]["events"]["spawn"]:
            pos, obj_type, index, player = spawn_event
            type_name = spawn_map[obj_type]
            if player not in spawn_events:
                spawn_events[player] = {}
            if round_idx not in spawn_events[player]:
                spawn_events[player][global_frame_idx] = defaultdict(list)
            spawn_events[player][global_frame_idx][type_name].append(pos)


In [88]:
frames[1].keys()

dict_keys(['p2Units', 'turnInfo', 'p1Stats', 'p1Units', 'p2Stats', 'events'])

Notes:
- Spawning -> moving happens on the same single turn!

In [104]:
class ReplayPlayer(Player):
    def __init__(self, spawn_events: dict):
        super().__init__()
        self.round_idx = 0
        self.spawn_events = [i[1] for i in sorted(spawn_events.items(), key=lambda x: x[0])]
    def deploy(self, game_state):
        spawn_events = self.spawn_events[self.round_idx]
        deployments = []
        for unit_type, positions in spawn_events.items():
            for pos in positions:
                if unit_type == "scout":
                    deployments.append((Scout(), tuple(pos)))
                elif unit_type == "demolisher":
                    deployments.append((Demolisher(), tuple(pos)))
                elif unit_type == "interceptor":
                    deployments.append((Interceptor(), tuple(pos)))
                elif unit_type == "turret":
                    deployments.append((Turret(), tuple(pos)))
                elif unit_type == "support":
                    deployments.append((Support(), tuple(pos)))
                elif unit_type == "wall":
                    deployments.append((Wall(), tuple(pos)))
        return deployments
    def upgrade(self, game_state):
        spawn_events = self.spawn_events[self.round_idx]
        upgrades = []
        for unit_type, positions in spawn_events.items():
            for pos in positions:
                if unit_type == "upgrade":
                    upgrades.append(tuple(pos))
        self.round_idx += 1
        return upgrades

In [108]:
rp1 = ReplayPlayer(spawn_events[1])
rp2 = ReplayPlayer(spawn_events[2])

game = TerminalGame(rp1, rp2)
print(game.render())

Turn: 0/100
Frame: 0

Player 1: ♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥                          
Player 2: ♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥                          

P1 Resources - Mobile: 5.0, Structure: 40.0
P2 Resources - Mobile: 5.0, Structure: 40.0

                          · ·                           
                        · · · ·                         
                      · · · · · ·                       
                    · · · · · · · ·                     
                  · · · · · · · · · ·                   
                · · · · · · · · · · · ·                 
              · · · · · · · · · · · · · ·               
            · · · · · · · · · · · · · · · ·             
          · · · · · · · · · · · · · · · · · ·           
        · · · · · · · · · · · · · · · · · · · ·         
      · · · · · · · · · · · · · · · · · · · · · ·       
    · · · · · · · · · · · · · · · · · · · · · · · ·     
  · · · · · · · · · · · · · · · · · · · · · · · · · ·   
· · · · · · ·

In [109]:
game.deploy_phase()
print(game.render())

Turn: 0/100
Frame: 0

Player 1: ♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥                          
Player 2: ♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥♥                          

P1 Resources - Mobile: 0.0, Structure: 1.0
P2 Resources - Mobile: 0.0, Structure: 0.0

                          u u                           
                        · u · ·                         
                      i · · · · i                       
                    · · · · · · · ·                     
                  · · · · · · · · · ·                   
                · · · · · · · · · · · ·                 
              · · · · · · · · · · · · · ·               
            i · · · · · · · · · · · · · · i             
          · · · · · · · · · · · · · · · · · ·           
        · · · · · · · · · · · · · · · · · · · ·         
      · · · · · · · · · · · · · · · · · · · · · ·       
    · · · · · · · · · · · · · · · · · · · · · · · ·     
  · · t · · · · · · · · · · · · · · · · · · · · t · ·   
· · · w · · · ·